In [35]:
!pip install flask

In [ ]:
import sqlite3 

creation = """CREATE TABLE IF NOT EXISTS "guardians" (
	"guardian_id"	INTEGER UNIQUE,
	"guardian_name"	TEXT,
	"guardian_contact"	INTEGER,
	PRIMARY KEY("guardian_id")
);

CREATE TABLE IF NOT EXISTS "students" (
	"student_id"	INTEGER UNIQUE,
	"stu_name"	TEXT,
	"gender"	TEXT CHECK('male' OR 'female'),
	"primary_sch"	TEXT,
	"guardian_id"	INTEGER,
	FOREIGN KEY("guardian_id") REFERENCES "guardians"("guardian_id"),
	PRIMARY KEY("student_id")
	
CREATE TABLE IF NOT EXISTS "results" (
	"student_id"	INTEGER,
	"subject"	TEXT,
	"al"	INTEGER CHECK("al" BETWEEN 1 AND 8),
	FOREIGN KEY("student_id") REFERENCES "students"("student_id"),
	PRIMARY KEY("student_id","subject")
);"""

with sqlite3.connect('ENROLMENT.db') as conn:
    cursor = conn.cursor()
    cursor.execute("BEGIN TRANSACTION")
    cursor.execute(creation)
    conn.commit()

In [12]:
import csv
import sqlite3

with open('GUARDIAN.csv') as f:
    reader = csv.reader(f)
    headers = next(reader)

    guardians = []
    for guardian in reader:
        guardians += (guardian, )

with open('RESULT.csv') as f:
    reader = csv.reader(f)
    headers = next(reader)

    results = []
    for result in reader:
        results += (result, )

with open('STUDENT.csv') as f:
    reader = csv.reader(f)
    headers = next(reader)

    students = []
    for student in reader:
        students += (student, )

insert_guardian = "INSERT INTO guardians VALUES (?, ?, ?)"
insert_result = "INSERT INTO results VALUES (?, ?, ?)"
insert_students = "INSERT INTO students VALUES (?, ?, ?, ?, ?)"

with sqlite3.connect('ENROLMENT.db') as conn:
    cursor = conn.cursor()
    cursor.execute("BEGIN TRANSACTION")

    for guardian in guardians:
        cursor.execute(insert_guardian, guardian)
    
    for result in results:
        cursor.execute(insert_result, result)
    
    for student in students:
        cursor.execute(insert_students, student)

    conn.commit()

In [13]:
with sqlite3.connect('ENROLMENT.db') as conn:
    cursor = conn.cursor()

    cursor.execute("UPDATE guardians SET guardian_contact = 'see@see.com' WHERE guardian_id = 900005")

    conn.commit()

In [44]:
import flask, sqlite3

query = " SELECT primary_sch, ?, SUM(al) / ? FROM results INNER JOIN students \
    ON results.student_id = students.student_id WHERE primary_sch = ?"

with sqlite3.connect(r'C:\Users\eugen\Desktop\Computing_notes\Computing-notes\Journey_to_A\Web_application\Web Applications Practice 3\ENROLMENT.db') as conn:
    cursor = conn.cursor()
    schools = list(set(\
        cursor.execute("SELECT primary_sch FROM students").fetchall()))
    data = []
    for school in schools:
        number_students = cursor.execute("SELECT COUNT(student_id) FROM students WHERE primary_sch = ?", school).fetchall()[0][0]
        data += cursor.execute(query, (int(number_students), int(number_students), school[0])).fetchall()
    data.sort(key= lambda x: x[0], reverse= False)

data

[('Boon Day Park Primary', 4, 9.25),
 ('Lurang Primary', 4, 10.75),
 ('Shuqu Primary', 3, 8.333333333333334)]